In [1]:
%cd /qbio/junsoopablo/2022_bioinfo/project/binfo1-datapack1

/qbio/junsoopablo/2022_bioinfo/project/binfo1-datapack1


In [2]:
import numpy as np
import pickle
from sklearn import decomposition
from scipy.stats import spearmanr
from functools import reduce
import matplotlib.ticker as ticker
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio import SeqIO
import matplotlib.colors as mcolors
import pylab as pl
from collections import Counter
from matplotlib import colors as colors

plt.style.use('default')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'
sns.set_style("whitegrid")

# fig.patch.set_facecolor('white')
%matplotlib inline

In [3]:
trgenename = pd.read_csv('/qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/references/Mouse-SCV2.transcript-genenames.txt', sep='\t', names=['transcript_id', 'gene_name'], index_col=0)
codinglengths = pd.read_csv('/qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/references/transcript-coding-lengths.txt', sep='\t',
                            names='transcript_id gene_id utr5 cds utr3 noncoding'.split())

repisoform = pickle.load(open('/qbio/junsoopablo//02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/ribojun/references/representative-isoforms.pickle','rb'))


codinglengths['genesize'] = codinglengths['utr5'] + codinglengths['cds'] + codinglengths['utr3']
codinglengths_gene = pd.merge(codinglengths, trgenename, how='inner', left_on='transcript_id', right_index=True)
codinglengths_gene = codinglengths_gene.loc[codinglengths_gene['transcript_id'].isin(repisoform['transcript_id'])]

cdslen_dic = codinglengths_gene.set_index('transcript_id')['cds'].to_dict()
utr3len_dic = codinglengths_gene.set_index('transcript_id')['utr3'].to_dict()
gene_dic = codinglengths_gene.copy(deep=True)
gene_dic['gene_id'] = gene_dic['gene_id'].str.split('.',expand=True)[0]
gene_dic = gene_dic.set_index('transcript_id')['gene_id'].to_dict()

genelen_dic = codinglengths_gene.set_index('transcript_id')['genesize'].to_dict()
genesym_dic = codinglengths_gene[['gene_id', 'gene_name']]
genesym_dic['gene_id'] = genesym_dic['gene_id'].str.split('.',expand=True)[0]
genesym_dic = genesym_dic.set_index('gene_id')['gene_name'].to_dict()


codinglengths_gene.head()

/tmp/ipykernel_1160685/1395101429.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genesym_dic['gene_id'] = genesym_dic['gene_id'].str.split('.',expand=True)[0]


,transcript_id,gene_id,utr5,cds,utr3,noncoding,genesize,gene_name
0,ENSMUST00000000001.5,ENSMUSG00000000001.5,141,1065,2056,0,3262,Gnai3
1,ENSMUST00000000003.14,ENSMUSG00000000003.16,140,525,237,0,902,Pbsn
2,ENSMUST00000000010.9,ENSMUSG00000020875.10,85,753,1736,0,2574,Hoxb9
3,ENSMUST00000000028.14,ENSMUSG00000000028.16,313,1701,129,0,2143,Cdc45
5,ENSMUST00000000049.6,ENSMUSG00000000049.12,51,1038,101,0,1190,Apoh


In [4]:
codinglengths_gene.loc[codinglengths_gene['gene_name']=='Rpl26']

,transcript_id,gene_id,utr5,cds,utr3,noncoding,genesize,gene_name
75713,ENSMUST00000167436.3,ENSMUSG00000060938.15,128,438,40,0,606,Rpl26


# Convert genome bam to transcript bam

In [5]:
! java -jar /qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/ribojun/contrib/ubu.jar \
    sam-xlate --bed /qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/ribojun/references/Mouse-SCV2.bed12 \
        --in RNA-siLuc.bam --out RNA-siLuc_tmp.bam --single --reverse
! samtools view -h RNA-siLuc_tmp.bam | \
        awk -vOFS='\t' '/^@/ { print $0; } \
        /^[^@]/ { $2 = xor($2, 8); print }' | \
    samtools sort -@ 10 --write-index -o filtered-RNA-siLuc_transcriptome.bam -

Starting Fri Jun 03 18:21:55 KST 2022
Parsing arguments
bedFile: /qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/ribojun/references/Mouse-SCV2.bed12
readOffset: 25
inputAlignmentFile: RNA-siLuc.bam
outputAlignmentFile: RNA-siLuc_tmp.bam
orderFastaFile: not specified
positiveStrandReportingOnly: false
xgtags: false
single: true

Skipping isoform header order determination
Building read index
Loaded 10001 isoforms.
Loaded 20001 isoforms.
Loaded 30001 isoforms.
Loaded 40001 isoforms.
Loaded 50001 isoforms.
Loaded 60001 isoforms.
Loaded 70001 isoforms.
Loaded 80001 isoforms.
Loaded 90001 isoforms.
Loaded 100001 isoforms.
Loaded 110001 isoforms.
Loaded 120001 isoforms.
Loaded 130001 isoforms.
Loaded 140001 isoforms.
Converting
Reading file: RNA-siLuc.bam
Writing to file: RNA-siLuc_tmp.bam
Writing header
Processing single end reads
Indexing isoforms for: chr1
Processed 1 reads.
Exception in thread "main" java.lang.IllegalStateException: Case statement didn't deal with cigar 

In [6]:
! java -jar /qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/ribojun/contrib/ubu.jar \
    sam-xlate --bed /qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/ribojun/references/Mouse-SCV2.bed12 \
        --in RNA-siLin28a.bam --out RNA-siLin28a_tmp.bam --single --reverse
! samtools view -h RNA-siLin28a_tmp.bam | \
        awk -vOFS='\t' '/^@/ { print $0; } \
        /^[^@]/ { $2 = xor($2, 8); print }' | \
    samtools sort -@ 10 --write-index -o filtered-RNA-siLin28a_transcriptome.bam -

Starting Fri Jun 03 18:22:52 KST 2022
Parsing arguments
bedFile: /qbio/junsoopablo/02_Projects/01_SCV2-Ribo/07_Chang/scv2-riboseq/ribojun/references/Mouse-SCV2.bed12
readOffset: 25
inputAlignmentFile: RNA-siLin28a.bam
outputAlignmentFile: RNA-siLin28a_tmp.bam
orderFastaFile: not specified
positiveStrandReportingOnly: false
xgtags: false
single: true

Skipping isoform header order determination
Building read index
Loaded 10001 isoforms.
Loaded 20001 isoforms.
Loaded 30001 isoforms.
Loaded 40001 isoforms.
Loaded 50001 isoforms.
Loaded 60001 isoforms.
Loaded 70001 isoforms.
Loaded 80001 isoforms.
Loaded 90001 isoforms.
Loaded 100001 isoforms.
Loaded 110001 isoforms.
Loaded 120001 isoforms.
Loaded 130001 isoforms.
Loaded 140001 isoforms.
Converting
Reading file: RNA-siLin28a.bam
Writing to file: RNA-siLin28a_tmp.bam
Writing header
Processing single end reads
Indexing isoforms for: chr1
Processed 1 reads.
Exception in thread "main" java.lang.IllegalStateException: Case statement didn't deal

# convert bam to rpf counts

In [7]:
!bedtools genomecov -ibam filtered-RNA-siLuc_transcriptome.bam -bg -5 > siLuc_RNA_transcriptome.bed
!head siLuc_RNA_transcriptome.bed

ENSMUST00000140995.2	188	189	1
ENSMUST00000140995.2	233	234	3
ENSMUST00000160777.8	910	911	4
ENSMUST00000160777.8	1859	1860	2
ENSMUST00000160777.8	2477	2478	2
ENSMUST00000160777.8	3558	3559	3
ENSMUST00000160777.8	3822	3823	1
ENSMUST00000160777.8	4158	4159	4
ENSMUST00000194886.2	318	319	2
ENSMUST00000194886.2	327	328	2


In [8]:
!bedtools genomecov -ibam filtered-RNA-siLin28a_transcriptome.bam -bg -5 > siLin28a_RNA_transcriptome.bed
!head siLin28a_RNA_transcriptome.bed

ENSMUST00000192973.2	1100	1101	1
ENSMUST00000192183.2	700	701	1
ENSMUST00000192183.2	755	756	1
ENSMUST00000192183.2	758	759	2
ENSMUST00000161581.2	103	104	1
ENSMUST00000195166.2	1904	1905	1
ENSMUST00000195166.2	1905	1906	27
ENSMUST00000195166.2	1906	1907	21
ENSMUST00000195166.2	1907	1908	27
ENSMUST00000195166.2	1908	1909	22
